In [1]:
pip install notebook


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyspark


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession

In [4]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("PySpark-jupyter-simpsons-task") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/02 16:56:29 WARN Utils: Your hostname, DESKTOP-J0ATGJO, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/02 16:56:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/02 16:56:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df_lines = spark.read.format("com.databricks.spark.csv").options(header='true') \
    .option("delimiter", ",").option("quote", '"') \
    .option("escape", '"').option("multiLine", True) \
    .load("/home/lior/pyspark/simpsons/simpsons_script_lines.csv")

df_lines.createOrReplaceTempView("lines_table")

df_lines.printSchema()
spark.sql("SELECT * FROM lines_table").show(5)

root
 |-- id: string (nullable = true)
 |-- episode_id: string (nullable = true)
 |-- number: string (nullable = true)
 |-- raw_text: string (nullable = true)
 |-- timestamp_in_ms: string (nullable = true)
 |-- speaking_line: string (nullable = true)
 |-- character_id: string (nullable = true)
 |-- location_id: string (nullable = true)
 |-- raw_character_text: string (nullable = true)
 |-- raw_location_text: string (nullable = true)
 |-- spoken_words: string (nullable = true)
 |-- normalized_text: string (nullable = true)
 |-- word_count: string (nullable = true)

+----+----------+------+--------------------+---------------+-------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|  id|episode_id|number|            raw_text|timestamp_in_ms|speaking_line|character_id|location_id|  raw_character_text|   raw_location_text|        spoken_words|     normalized_text|word_count|
+----+----------+------+-------------

In [6]:
spark.sql("SELECT int(word_count) AS wc, raw_text FROM lines_table WHERE TRY_CAST(word_count AS INT) > 0 ORDER BY wc ").show(10,False) 

[Stage 2:>                                                          (0 + 1) / 1]

+---+---------------------------------+
|wc |raw_text                         |
+---+---------------------------------+
|1  |Homer Simpson: Oh.               |
|1  |Homer Simpson: And?              |
|1  |Bart Simpson: Lewis?             |
|1  |Homer Simpson: (SHOCKED) Me?     |
|1  |Wendell Borton: Yayyyyyyyyyyyyyy!|
|1  |Lisa Simpson: Baboon!            |
|1  |Lisa Simpson: Yeah.              |
|1  |Lisa Simpson: No!                |
|1  |Homer Simpson: Oh.               |
|1  |Homer Simpson: Nuts.             |
+---+---------------------------------+
only showing top 10 rows


In [7]:
spark.sql("SELECT count(*) FROM lines_table WHERE TRY_CAST(timestamp_in_ms AS INT) > (10 * 60 * 1000)").show() 

[Stage 3:>                                                          (0 + 1) / 1]

+--------+
|count(1)|
+--------+
|   86678|
+--------+



# Ex 1.1

In [8]:
# Print the first 50 locations that contain the word "Springfield" (case-insensitive)
spark.sql("""
    SELECT DISTINCT
        raw_location_text
    FROM lines_table
    WHERE raw_location_text IS NOT NULL
      AND LOWER(raw_location_text) LIKE '%springfield%'
    LIMIT 50
""").show(50, truncate=False)


[Stage 6:>                                                          (0 + 1) / 1]

+---------------------------------------+
|raw_location_text                      |
+---------------------------------------+
|Springfield Police Station             |
|Springfield YMCA                       |
|SPRINGFIELD REPUBLICAN PARTY CASTLE    |
|Springfield                            |
|Springfield Pond                       |
|Springfield Mall                       |
|SPRINGFIELD ANIMAL REFUGE              |
|Springfield Community Theater          |
|SPRINGFIELD - THE PAST                 |
|PRIMORDIAL SPRINGFIELD                 |
|SPRINGFIELD HOSPITAL HALLWAY           |
|SPRINGFIELD ELEMENTARY SCHOOL BUS      |
|Springfield Elementary School Band Room|
|Springfield Opry House                 |
|Springfield Swap Meet                  |
|SPRINGFIELD PIER                       |
|Springfield Air and Space Museum       |
|Springfield Trade Center               |
|DREAMLIKE SUBURBAN SPRINGFIELD STREET  |
|SPRINGFIELD MONUMENT PARK              |
|Lake Springfield                 

# Ex 1.2

In [9]:
#Load Locations csv
df_locations = spark.read.option("header", True).csv(
    "/home/lior/pyspark/simpsons/simpsons_locations.csv"
)
df_locations.createOrReplaceTempView("locations_table")

In [10]:
df_locations.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- normalized_name: string (nullable = true)



In [11]:
# Print 20 quotes located in places that contain "Jerusalem" (case-insensitive)
# Uses JOIN as required by the assignment
spark.sql("""
    SELECT
        l.raw_text,
        loc.name AS location_name
    FROM lines_table l
    JOIN locations_table loc
        ON TRY_CAST(l.location_id AS INT) = TRY_CAST(loc.id AS INT)
    WHERE LOWER(loc.name) LIKE '%jerusalem%'
    LIMIT 20
""").show(20, truncate=False)


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|raw_text                                                                                                                                                                                  |location_name         |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|(JERUSALEM ROYAL PALACE: ext. jerusalem royal palace - plaza - day)                                                                                                                       |JERUSALEM ROYAL PALACE|
|Bart Simpson: I'm bored. Send in my jester.                                                                                                            

# Ex 1.3

In [12]:
# Print the first 20 most used locations with the count of lines spoken in them
spark.sql("""
    SELECT
        raw_location_text AS location_name,
        COUNT(*) AS lines_spoken
    FROM lines_table
    WHERE raw_location_text IS NOT NULL
      AND TRIM(raw_location_text) != ''
    GROUP BY raw_location_text
    ORDER BY lines_spoken DESC
    LIMIT 20
""").show(20, truncate=False)


[Stage 12:>                                                         (0 + 1) / 1]

+-------------------------------+------------+
|location_name                  |lines_spoken|
+-------------------------------+------------+
|Simpson Home                   |35059       |
|Springfield Elementary School  |7092        |
|Moe's Tavern                   |4628        |
|Springfield Nuclear Power Plant|3594        |
|Kwik-E-Mart                    |1476        |
|First Church of Springfield    |1416        |
|Simpson Living Room            |1378        |
|Springfield Street             |1334        |
|Springfield                    |1314        |
|Simpson Car                    |1239        |
|Flanders Home                  |1166        |
|Street                         |1124        |
|Springfield Town Hall          |1103        |
|Springfield Retirement Castle  |1049        |
|Burns Manor                    |998         |
|Springfield Mall               |833         |
|Simpson Kitchen                |816         |
|Courtroom                      |813         |
|Bart's Treeh

# Ex 1.4

In [13]:
# Load episodes table
df_episodes = spark.read.option("header", True).csv(
    "/home/lior/pyspark/simpsons/simpsons_episodes.csv"
)

df_episodes.createOrReplaceTempView("episodes_table")
df_episodes.printSchema()


root
 |-- id: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- imdb_rating: string (nullable = true)
 |-- imdb_votes: string (nullable = true)
 |-- number_in_season: string (nullable = true)
 |-- number_in_series: string (nullable = true)
 |-- original_air_date: string (nullable = true)
 |-- original_air_year: string (nullable = true)
 |-- production_code: string (nullable = true)
 |-- season: string (nullable = true)
 |-- title: string (nullable = true)
 |-- us_viewers_in_millions: string (nullable = true)
 |-- video_url: string (nullable = true)
 |-- views: string (nullable = true)



In [14]:
# Find the seasons with the highest average IMDb rating
spark.sql("""
    SELECT
        TRY_CAST(season AS INT) AS season,
        COUNT(*) AS episode_count,
        AVG(TRY_CAST(imdb_rating AS DOUBLE)) AS avg_imdb_rating
    FROM episodes_table
    WHERE TRY_CAST(season AS INT) IS NOT NULL
      AND TRY_CAST(imdb_rating AS DOUBLE) IS NOT NULL
    GROUP BY TRY_CAST(season AS INT)
    ORDER BY avg_imdb_rating DESC
""").show(50, truncate=False)


+------+-------------+------------------+
|season|episode_count|avg_imdb_rating   |
+------+-------------+------------------+
|5     |22           |8.336363636363636 |
|7     |25           |8.324             |
|6     |25           |8.312             |
|4     |22           |8.268181818181818 |
|8     |25           |8.219999999999999 |
|3     |24           |8.154166666666665 |
|2     |22           |8.04090909090909  |
|9     |25           |7.8439999999999985|
|1     |13           |7.807692307692307 |
|10    |23           |7.569565217391306 |
|12    |21           |7.361904761904761 |
|11    |22           |7.290909090909093 |
|13    |22           |7.140909090909091 |
|14    |22           |7.077272727272727 |
|15    |22           |7.045454545454546 |
|16    |21           |7.042857142857143 |
|18    |22           |7.0               |
|19    |20           |6.935             |
|20    |21           |6.895238095238096 |
|17    |22           |6.863636363636362 |
|25    |22           |6.8318181818

# Ex 2 

In [15]:
# Load characters table
df_characters = spark.read.option("header", True).csv(
    "/home/lior/pyspark/simpsons/simpsons_characters.csv"
)

# Create temp view for SQL
df_characters.createOrReplaceTempView("characters_table")

df_characters.printSchema()


root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- normalized_name: string (nullable = true)
 |-- gender: string (nullable = true)



In [16]:
# Which character talks the most in The Simpsons?
spark.sql("""
    SELECT
        c.name AS character_name,
        COUNT(*) AS lines_spoken
    FROM lines_table l
    JOIN characters_table c
        ON TRY_CAST(l.character_id AS INT) = TRY_CAST(c.id AS INT)
    WHERE c.name IS NOT NULL
    GROUP BY c.name
    ORDER BY lines_spoken DESC
    LIMIT 10
""").show(10, truncate=False)


[Stage 21:>                                                         (0 + 1) / 1]

+-------------------+------------+
|character_name     |lines_spoken|
+-------------------+------------+
|Homer Simpson      |30108       |
|Marge Simpson      |14265       |
|Bart Simpson       |13969       |
|Lisa Simpson       |11641       |
|C. Montgomery Burns|3207        |
|Moe Szyslak        |2864        |
|Seymour Skinner    |2443        |
|Ned Flanders       |2145        |
|Grampa Simpson     |1957        |
|Milhouse Van Houten|1911        |
+-------------------+------------+

